In [1]:
import csv
import xml.etree.ElementTree as ET
from xml.dom import minidom
import shutil
from PIL import Image
import cv2
import os
import random
import duckdb 
import pandas as pd
import sqlalchemy
%load_ext sql 
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


In [2]:
%sql duckdb:///:memory:

In [3]:
%sql create table train as select image_path,name,xmax,xmin,ymax,ymin from train.csv

,Count
0,19950


In [7]:
%sql select name , count(*) from train group by 1 

,name,count_star()
0,GARBAGE,8597
1,BAD_BILLBOARD,1555
2,SAND_ON_ROAD,748
3,GRAFFITI,1124
4,POTHOLES,2625
5,CLUTTER_SIDEWALK,2253
6,CONSTRUCTION_ROAD,2730
7,BROKEN_SIGNAGE,83
8,UNKEPT_FACADE,127
9,FADED_SIGNAGE,107


In [ ]:
# convert the given train.csv into PASCAL VOC xml file for each image.


import csv
import xml.etree.ElementTree as ET
def csv_to_bacal_voc_xml(csv_file, adjustment_factor):
    xml_strings = []
    prev_image_path = ""
    root = ET.Element('annotation')
    folder = ET.SubElement(root, 'folder')
    folder.text = "my-project-name"
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['image_path'] != prev_image_path:
                xml_strings.append({"image_path": prev_image_path, "xml": prettify(root)})
                root.clear()
                folder = ET.SubElement(root, 'folder')
                folder.text = "my-project-name"
                filename = ET.SubElement(root, 'filename')
                filename.text = row['image_path']

                path = ET.SubElement(root, 'path')
                path.text = '/my-project-name/' + row['image_path']

                source = ET.SubElement(root, 'source')
                database = ET.SubElement(source, 'database')
                database.text = 'Unspecified'

                size = ET.SubElement(root, 'size')
                width = '1920'
                height = '1080'
                width_tag = ET.SubElement(size, 'width')
                width_tag.text = width
                height_tag = ET.SubElement(size, 'height')
                height_tag.text = height
                depth = ET.SubElement(size, 'depth')
                depth.text = '3'
                prev_image_path = row['image_path']
            obj = ET.SubElement(root, 'object')
            ET.SubElement(obj, 'name').text = row['name']
            ET.SubElement(obj, 'pose').text = 'Unspecified'
            ET.SubElement(obj, 'truncated').text = '0'
            ET.SubElement(obj, 'difficult').text = '0'
            bndbox = ET.SubElement(obj, 'bndbox')
            ET.SubElement(bndbox, 'xmin').text = str(int(float(row['xmin']) * adjustment_factor))
            ET.SubElement(bndbox, 'ymin').text = str(int(float(row['ymin']) * adjustment_factor))
            ET.SubElement(bndbox, 'xmax').text = str(int(float(row['xmax']) * adjustment_factor))
            ET.SubElement(bndbox, 'ymax').text = str(int(float(row['ymax']) * adjustment_factor))
    xml_strings.append({"image_path": prev_image_path, "xml": prettify(root)})
    return xml_strings


def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")



In [ ]:
train_csv = 'train.csv'

In [ ]:
# converted xml_files
xml_files = csv_to_bacal_voc_xml(train_csv, 2)


In [ ]:
import os
def save_xml_files(xml_list,directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    for item in xml_list:
        if item["image_path"] != "":
            file_name, file_extension = os.path.splitext(item["image_path"])
            new_file_name = file_name + ".xml"
            path=os.path.join(directory,new_file_name)
            with open(path, "w") as f:
                f.write(item["xml"])

In [ ]:


annotations_train_path = "image_spool/preprocessing/dataset/train/Annotations"
if os.path.exists(annotations_train_path):
    shutil.rmtree(annotations_train_path)


In [ ]:
# save annotations to Annotations folder.
save_xml_files(xml_files,annotations_train_path)

In [ ]:
def copy_images(xml_list,src_dir,dst_dir):
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for item in xml_list:
        if item["image_path"] != "":
            src_path=os.path.join(src_dir,item["image_path"])
            dst_path=os.path.join(dst_dir,item["image_path"])
            shutil.copy2(src_path,dst_path)


In [ ]:

images_train_path = "image_spool/preprocessing/dataset/train/Images"
if os.path.exists(images_train_path):
    shutil.rmtree(images_train_path)


In [ ]:
# copy images from raw dataset to Images folder.
base_image_path = 'image_spool/preprocessing/raw_dataset'
copy_images(xml_files,base_image_path,images_train_path)

In [ ]:
# test dataset path
test_images_path = "image_spool/preprocessing/dataset/test/Images"
test_annotation_path = "image_spool/preprocessing/dataset/test/Annotations"



In [ ]:
# list dir of images_train_path 
image_train_base = os.listdir(images_train_path)
annotation_train_base = os.listdir(annotations_train_path)
print('Train images: ',len(image_train_base))
print('Train annotations: ',len(annotation_train_base))

In [ ]:
random.shuffle(image_train_base)

In [ ]:
images_20_percent = image_train_base[:int(len(image_train_base)*0.2)]

In [ ]:

if not os.path.exists(test_images_path):
         os.makedirs(test_images_path)
   
if not os.path.exists(test_annotation_path):
         os.makedirs(test_annotation_path)

In [ ]:
# copy images to test_images_path
for image in images_20_percent:

    
    src_path=os.path.join(images_train_path,image)
    if os.path.exists(src_path):
       print(image)
       img_file_name, img_file_extension = os.path.splitext(image)
       new_file_name = img_file_name + img_file_extension
       shutil.copy2(src_path,test_images_path + new_file_name)
       # remove image from train
       os.remove(src_path)

       # copy annotations to test_annotation_path
       file_name, file_extension = os.path.splitext(image)
       new_file_name = file_name + ".xml"
       src_path=os.path.join(annotations_train_path,new_file_name)
       if os.path.exists(src_path):
              print(new_file_name)
              shutil.copy2(src_path,test_annotation_path + new_file_name)
                # remove annotation from train
              os.remove(src_path)
    